In [2]:
import pandas as pd

In [ ]:
# lemmas의 개수가 달라서 컬럼 숫자가 다름. 그로인한 오류를 방지하기 위해 일단 컬럼수를 50으로 잡아줌
bsl_df = pd.read_csv("./raw_data/BSL_1.01_lemmatized_for_research.csv", header=None, names=range(50))
# 컬럼 전체가 NaN인 컬럼은 떨궈줌
bsl_df = bsl_df.dropna(axis=1, how='all') 
# NaN 값을 '' 로 바꾼후 word1;word2;word3 로 컬럼 저장
bsl_df = bsl_df.fillna('')
bsl_df['Lemmas'] = bsl_df[list(range(1, 14))].agg(';'.join, axis=1).str.rstrip(';')
bsl_df = bsl_df.rename(columns={0: "Word"})
# Word, Lemmas 컬럼 빼고 다 떨궈줌
bsl_df = bsl_df.drop(bsl_df.columns.difference(['Word', 'Lemmas']), 1)
bsl_df.to_csv("bsl_with_lemmas.csv", index=False)

# row = bsl_df[~bsl_df[13].isnull()]

bsl_df
# bsl_df.to_csv("bsl_with_lemmas.csv", index=False)

In [ ]:
tfl_df = pd.read_csv("./raw_data/toeflw.csv")
tfl_df = tfl_df.drop(tfl_df.columns.difference(['Word']), 1)

tfl_df = tfl_df.sort_values('Word')
tfl_df
# tfl_df.to_csv("toefl_with_lemmas.csv", index=False)

In [ ]:
# lemmas의 개수가 달라서 컬럼 숫자가 다름. 그로인한 오류를 방지하기 위해 일단 컬럼수를 50으로 잡아줌
tsl_df = pd.read_csv("./raw_data/TSL_1.1_lemmatized_for_research.csv", header=None, names=range(50))
# 컬럼 전체가 NaN인 컬럼은 떨궈줌
tsl_df = tsl_df.dropna(axis=1, how='all') 
# NaN 값을 '' 로 바꾼후 word1;word2;word3 로 컬럼 저장
tsl_df = tsl_df.fillna('')
tsl_df['Lemmas'] = tsl_df[list(range(1, 11))].agg(';'.join, axis=1).str.rstrip(';')
tsl_df = tsl_df.rename(columns={0: "Word"})
# # Word, Lemmas 컬럼 빼고 다 떨궈줌
tsl_df = tsl_df.drop(tsl_df.columns.difference(['Word', 'Lemmas']), 1)

# # row = tsl_df[~tsl_df[13].isnull()]

tsl_df
# tsl_df.to_csv("tsl_with_lemmas.csv", index=False)

In [ ]:
lemma_df = pd.read_csv("./raw_data/lemma.en.txt", sep="->", header=None, names=["Head", "Lemmas"])
lemma_df['Head'] = lemma_df['Head'].apply(lambda w: w.split("/")[0]).str.strip()
lemma_df['Lemmas'] = lemma_df['Lemmas'].str.strip()

a = lemma_df.loc[84479]["Head"]
b = lemma_df.loc[84479]["Lemmas"]
print([a])
print([b])

# lemma_df

# lemma_df.to_csv("entire_lemmas.csv", index=False)

In [ ]:
# awsl_df = pd.read_csv("./words_refined/Dang,Coxhead.csv")
# awsl_df['Word'] = awsl_df['Word'].str.strip()
# awsl_df['Lemmas'] = awsl_df['Lemmas'].str.strip().str.strip(';').str.replace(' ', '')

# awsl_df

# awsl_df.to_csv("awsl_with_lemmas.csv", index=False)

In [55]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 100)

ox = pd.read_csv("../raw_data/oxford_3000.txt", header=None, sep=' ', names=range(50))
ox = ox[ox[14].isnull()]
ox = ox.dropna(axis=1, how='all') 
ox = ox.fillna('')
ox
# get normal words
normal_condition = ox[1].str.contains(".", regex=False) & ~ox[0].str.contains("[^a-zA-Z]")
normal = ox[normal_condition]
normal = normal[[0]]
normal

# exceptions
ex = ox[~normal_condition]
ex
# add levels to normal
levels = ox[ox[0].isin(['A1', 'A2', 'B1', 'B2'])]
levels
ex = ex[~ex[0].isin(['A1', 'A2', 'B1', 'B2'])]
ex
normal = normal.append(levels[[0]])
normal
normal.sort_index(inplace=True)

### exceptions 처리
numbers = ex[ex[1] == 'number']
numbers
normal = normal.append(numbers[[0]])
normal
ex = ex[~(ex[1] == 'number')]
ex

ones = ex[ex[0].str.len() == 1]
ex = ex[~(ex[0].str.len() == 1)]
ex = ex[~(ex[0].str.len() == 0)]
ex

ex[0] = ex[0].apply(lambda x: x[:-1] if x[-1].isdigit() else x)
ex
ex = ex.drop_duplicates(subset=0)
ex
# ex = ex.drop([1, 351, 438, 930, 517, 912])
ex = ex.drop([1, 350, 434, 929, 516, 911, 670])
ex

compounds = ex[~ex[1].str.contains("[^a-zA-Z]")]
compounds
compounds = compounds[(compounds[1] !='') & (compounds[1] != 'modal')].drop([779, 2846])
compounds
compounds[0] = compounds[0] + ' ' + compounds[1]
compounds
ex.update(compounds)
ex

# combine to normal
normal = normal.append(ex[[0]])
normal
normal[0] = normal[0].str.lower()
normal
# drop duplicates to easy version
normal.sort_index(inplace=True)
normal.drop_duplicates(subset=0, inplace=True)

### slice by level
normal = normal.rename(columns={0:'Word'})
normal
normal.reset_index(inplace=True)
normal
normal = normal.drop(columns='index')
normal
indices = normal[normal.Word.isin(['a1', 'a2', 'b1', 'b2'])]
indices
a1 = normal.loc[1:895]
a2 = normal.loc[897:1686]
b1 = normal.loc[1688:2376]
b2 = normal.loc[2378:]

a1['Level'] = 'A1'
a2['Level'] = 'A2'
b1['Level'] = 'B1'
b2['Level'] = 'B2'

final_df_3000 = pd.concat([a1, a2, b1, b2])
final_df_3000['Version'] = 3000
final_df_3000

# final_df_3000.to_csv("ox_3000_american.csv", index=False)


/var/folders/qz/yv1t73k93pv_4cwl9gbygk0w0000gn/T/ipykernel_40470/3862471014.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a1['Level'] = 'A1'
/var/folders/qz/yv1t73k93pv_4cwl9gbygk0w0000gn/T/ipykernel_40470/3862471014.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a2['Level'] = 'A2'
/var/folders/qz/yv1t73k93pv_4cwl9gbygk0w0000gn/T/ipykernel_40470/3862471014.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [86]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 100)

ox = pd.read_csv("../raw_data/oxford_5000.txt", header=None, sep=' ', names=range(50))
ox = ox[ox[14].isnull()]
ox = ox.dropna(axis=1, how='all') 
ox
ox = ox.fillna('')
ox
# get normal words
normal_condition = ox[1].str.contains(".", regex=False) & ~ox[0].str.contains("[^a-zA-Z]")
normal = ox[normal_condition]
normal = normal[[0]]
normal

# exceptions
ex = ox[~normal_condition]
ex
# add levels to normal
levels = ox[ox[0].isin(['B2', 'C1'])]
levels
ex = ex[~ex[0].isin(['B2', 'C1'])]
ex
normal = normal.append(levels[[0]])
normal
normal.sort_index(inplace=True)

### exceptions 처리
numbers = ex[ex[1] == 'number']
numbers
normal = normal.append(numbers[[0]])
normal
ex = ex[~(ex[1] == 'number')]
ex

ones = ex[ex[0].str.len() == 1]
ex = ex[~(ex[0].str.len() == 1)]
ex = ex[~(ex[0].str.len() == 0)]
ex

ex[0] = ex[0].apply(lambda x: x[:-1] if x[-1].isdigit() else x)
ex
ex = ex.drop_duplicates(subset=0)
ex

# combine to normal
normal = normal.append(ex[[0]])
normal
normal[0] = normal[0].str.lower()
normal
# drop duplicates to easy version
normal.sort_index(inplace=True)
normal.drop_duplicates(subset=0, inplace=True)
normal

### slice by level
normal = normal.rename(columns={0:'Word'})
normal
normal.reset_index(inplace=True)
normal
normal = normal.drop(columns='index')
normal
indices = normal[normal.Word.isin(['b2', 'c1'])]
indices
b2 = normal.loc[1:700]
c1 = normal.loc[702:]
b2['Level'] = 'B2'
c1['Level'] = 'C1'

final_df_5000 = pd.concat([b2, c1])
final_df_5000['Version'] = 5000
final_df_5000

# final_df_5000.to_csv("ox_5000_american.csv", index=False)


/var/folders/qz/yv1t73k93pv_4cwl9gbygk0w0000gn/T/ipykernel_40470/963181386.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b2['Level'] = 'B2'
/var/folders/qz/yv1t73k93pv_4cwl9gbygk0w0000gn/T/ipykernel_40470/963181386.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c1['Level'] = 'C1'
